In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from wdtools import *

In [3]:
import glob

In [4]:
wd_all = get_all_wd(num=6)

In [5]:
sa_gdf_all = get_all_SA(num=6)

In [6]:
IDstoRev = wd_all[~wd_all.wetdet_delin_number.isin(sa_gdf_all.wdID.unique())].wetdet_delin_number.unique()

In [7]:
wd_all.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot', 'SetID'],
      dtype='object')

In [8]:
set6IDs = wd_all[(wd_all.wetdet_delin_number.isin(IDstoRev))&(wd_all.IDyear=='2022')&(wd_all.SetID == 6)].wetdet_delin_number.unique()

In [9]:
set6IssueIDs = pd.read_csv(outpath + '\\to_review\\Set006_Mapping_Issues.csv').wetdet_delin_number.unique()

In [10]:
[wid for wid in set6IDs if wid not in set6IssueIDs]

[]

In [11]:
wd_all[(wd_all.wetdet_delin_number.isin(IDstoRev))&(wd_all.IDyear=='2022')&(wd_all.SetID != 6)][['wetdet_delin_number', 'county', 'trsqq', 'parcel_id','is_batch_file',
       'status_name', 'missinglot', 'notes', 'SetID']]

,wetdet_delin_number,county,trsqq,parcel_id,is_batch_file,status_name,missinglot,notes,SetID


In [12]:
wd_all[(wd_all.wetdet_delin_number.isin(IDstoRev))&(wd_all.IDyear!='2022')][['wetdet_delin_number', 'county', 'trsqq', 'parcel_id','is_batch_file',
       'status_name', 'missinglot', 'notes', 'SetID']]

,wetdet_delin_number,county,trsqq,parcel_id,is_batch_file,status_name,missinglot,notes,SetID
1147,WD2021-0703,Lake,38S20E09,Many,N,Site Visit Needed,Y,Many,1
2491,WD2020-0389,Klamath,30S05E00,100p,N,Withdrawn by Applicant,N,Partial,3
2492,WD2020-0389,Klamath,30S06E00,200p,N,Withdrawn by Applicant,N,Partial,3
2493,WD2020-0389,Klamath,31S06E00,100p,N,Withdrawn by Applicant,N,Partial,3
2494,WD2020-0389,Klamath,31S07E00,200p,N,Withdrawn by Applicant,N,Partial,3
...,...,...,...,...,...,...,...,...,...
4834,WD2022-0656,Marion,08S01E31,100p,N,Hold,N,Partial,6
4835,WD2022-0656,Marion,09S02E16,"100p,1200p,1300p,1600p,ROW",N,Hold,N,"ROW, Partial",6
4950,WD2022-0656,Wasco,08S08E00,200p,N,Hold,N,Partial,6
4951,WD2022-0656,Wasco,08S09E00,100p,N,Hold,N,Partial,6


In [68]:
url = wd_all.loc[wd_all.wetdet_delin_number == 'WD2022-0158', 'DecisionLink'].values[0]
if str(url) == 'nan':
    print('Decision link is not available')
else:
    webbrowser.open(url)

Decision link is not available


file = outpath + '\\to_review\\code2.txt'
with open(file) as f:
    code2 = f.readlines()
code2IDs = code2[0].split(', ')
wd_all[wd_all.wetdet_delin_number.isin(code2IDs)].SetID.unique()
gdf = gpd.read_file(outpath + '\\final\\Set001_mapped_wd.shp')
gdf.loc[gdf.wdID.isin(code2IDs), 'code'] = 2
gdf.to_file(outpath + '\\final\\Set001_mapped_wd.shp')

gdf = gpd.read_file(outpath + '\\matched\\matched_records_Set005.shp')
gdf[gdf.wdID=='WD2019-0487'].plot()
gdf[gdf.wdID=='WD2019-0233'].plot()
ngdf = gpd.read_file(outpath + '\\final\\mapped_wd_Set005.shp')
sgdf = gdf[gdf.wdID.isin(['WD2019-0233', 'WD2019-0487'])][['wdID', 'geometry']]
sgdf = sgdf.dissolve('wdID')
sgdf['wdID'] = sgdf.index
sgdf.reset_index(drop=True, inplace=True)
sgdf['code'] = 0
setgdf = pd.concat([ngdf, sgdf[['code', 'wdID', 'geometry']]])
setgdf.to_file(os.path.join(inpath, "output", "final", f"Set005_mapped_wd.shp"), index=False)

In [13]:
file = outpath + '\\to_review\\re_mapping.txt'
with open(file) as f:
    remapIDs = f.readlines()

file = outpath + '\\to_review\\questions.txt'
with open(file) as f:
    qsIDs = f.readlines()

In [16]:
qsIDs_tb = pd.read_excel(inpath + '\\reporting\\QAQC_questions.xlsx')

In [22]:
qsIDs = list(qsIDs_tb.wetdet_delin_number.unique())

In [24]:
qaqcIDs = remapIDs[0].split(', ') + qsIDs

In [26]:
len(unique(qaqcIDs))

327

In [27]:
check_cnt = wd_all[wd_all.wetdet_delin_number.isin(unique(qaqcIDs))][['county', 'wetdet_delin_number']].groupby(['wetdet_delin_number']).agg(lambda x: x.nunique()).reset_index().rename(columns={'wetdet_delin_number':'wdID', 'county':'count'})

In [28]:
check_cnt[check_cnt['count']>1]

,wdID,count
70,WD2019-0008,2


In [29]:
wd_all[wd_all.wetdet_delin_number.isin(check_cnt[check_cnt['count']>1].wdID.unique())][['wetdet_delin_number', 'county', 'trsqq', 'parcel_id', 'SetID']]

,wetdet_delin_number,county,trsqq,parcel_id,SetID
2093,WD2019-0008,Morrow,01N26E00,"3502,3500,3400.3300,3301,3600,3201,3200,2900,2700",2
2094,WD2019-0008,Morrow,01N26E00,"1100,1102,1200,1301,1500,2805",2
2095,WD2019-0008,Morrow,01N25E00,"1600,3100",2
4111,WD2019-0008,Umatilla,02N28E00,1900p,5


In [30]:
qaqc_df = wd_all[wd_all.wetdet_delin_number.isin(qaqcIDs)][['county', 'wetdet_delin_number']].groupby(['county']).agg(lambda x: x.nunique()).reset_index().rename(columns={'wetdet_delin_number':'QAQC_count'})

In [31]:
qaqc_df.QAQC_count.sum()

327

In [32]:
total_df = wd_all[['county', 'wetdet_delin_number']].groupby(['county']).agg(lambda x: x.nunique()).reset_index().rename(columns={'wetdet_delin_number':'total_count'})

In [33]:
total_df.total_count.sum()

3080

In [34]:
(327+1011)/3080

0.4344155844155844

In [113]:
781+650+906+869+443+267

3916

In [35]:
total_df.merge(qaqc_df, on='county', how='left').to_csv(inpath+'\\reporting\\mapping_wd_counts.csv', index=False)

In [36]:
rep_dat = pd.read_excel(inpath+'\\reporting\\mapping_wd_counts.xlsx')

In [37]:
rep_dat.columns

Index(['County', 'Note', 'P1_2017_22', 'P1_ToMap', 'P2_2008_16', 'P2_ToMap',
       'P3_1990_07', 'P3_ToMap'],
      dtype='object')

In [38]:
rep_dat.Note.unique()

array(['Participating', 'Non-Participating'], dtype=object)

In [39]:
rep_dat.P1_ToMap.sum()

1100

In [40]:
df = rep_dat[rep_dat.Note=='Participating']

In [41]:
df.P1_ToMap.sum()

327

In [42]:
total = df.P1_2017_22.sum() + df.P2_2008_16.sum() + df.P3_1990_07.sum()

In [43]:
1011/df.P1_2017_22.sum()

0.32878048780487806

In [44]:
total

16262

In [145]:
# hourly billing rate
hbr = 87.51

In [146]:
# ratio for QAQC
qaqc_r = 0.5

In [154]:
# counts per hour
cpr = 1/0.8

In [148]:
# budget
bdgt = 50000

In [162]:
remain_total = df.P2_2008_16.sum() + df.P3_1990_07.sum()

In [167]:
remain_total*0.7/2090*40

176.66794258373204

In [168]:
(df.P1_ToMap.sum() + remain_total*qaqc_r)/4

1750.625

In [170]:
(1751+177)/570

3.382456140350877

In [155]:
total * qaqc_r * hbr/cpr/bdgt

11.384700960000002

In [156]:
mapped = df.P1_2017_22.sum() - df.P1_ToMap.sum()

In [158]:
mapped

2666

In [157]:
mapped / total

0.16394047472635592

In [150]:
bdgt/hbr

571.3632727688264

In [45]:
gdf = gpd.read_file(r'X:\Data\State\ORCnty2015.shp')
gdf = gdf[['altName', 'geometry']]
gdf.rename(columns={'altName':'County'}, inplace=True)

In [46]:
gdf = gdf.merge(rep_dat, on='County', how='left')

In [47]:
gdf.loc[:, 'Tot_ToMap'] =  gdf.P1_ToMap + gdf.P2_ToMap + gdf.P3_ToMap

In [48]:
gdf.loc[:, 'Tot_Count'] =  gdf.P1_2017_22 + gdf.P2_2008_16 + gdf.P3_1990_07

In [49]:
gdf.loc[:, 'QAQC_Ratio'] = gdf.Tot_ToMap / gdf.Tot_Count

In [50]:
gdf.loc[:, 'Auto_Comp'] = gdf.Tot_Count - gdf.Tot_ToMap

In [51]:
gdf.to_file(inpath+'\\reporting\\WD_Counts.shp')

In [212]:
os.remove(inpath+"\\reporting\\WD_Counts.zip")

In [207]:
file_name_list = glob.glob(inpath+"\\reporting\\WD_Counts.*")

In [193]:
zip_file_name = inpath+"\\reporting\\WD_Counts.zip"

In [208]:
file_name_list

['L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping\\reporting\\WD_Counts.cpg',
 'L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping\\reporting\\WD_Counts.dbf',
 'L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping\\reporting\\WD_Counts.prj',
 'L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping\\reporting\\WD_Counts.shp',
 'L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping\\reporting\\WD_Counts.shx']

In [213]:
file_compress(file_name_list, zip_file_name)

 *** Input File name passed for zipping - ['L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping\\reporting\\WD_Counts.cpg', 'L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping\\reporting\\WD_Counts.dbf', 'L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping\\reporting\\WD_Counts.prj', 'L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping\\reporting\\WD_Counts.shp', 'L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping\\reporting\\WD_Counts.shx']
 *** out_zip_file is - L:\NaturalResources\Wetlands\Local Wetland Inventory\WAPO\EPA_2022_Tasks\Task 1 WD Mapping\reporting\WD_Counts.zip
 *** Processing file L:\NaturalResources\Wetlands\Local Wetland Inventory\WAPO\EPA_2022_Tasks\Task 1 WD Mapping\reporting\WD_Counts.cpg
 *** Processing file L:\NaturalResources\Wetlands\Loc